In [85]:
import pandas as pd

In [125]:
csv_file = "Shmoodify (1).csv"
all_songs_df = pd.read_csv(csv_file, encoding="latin-1")
all_songs_df.head()

,.,Song,Artist,Year,Lyrics,Source,Schmood
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0,saucy
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love you...,1.0,love
2,4,you were on my mind,we five,1965,when i woke up this morning you were on my min...,1.0,sad
3,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss ...,1.0,love
4,6,downtown,petula clark,1965,when youre alone and life is making you lonely...,1.0,sad


In [126]:
#keep training dataset
train_songs = all_songs_df.dropna()
train_songs.head()

,.,Song,Artist,Year,Lyrics,Source,Schmood
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0,saucy
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love you...,1.0,love
2,4,you were on my mind,we five,1965,when i woke up this morning you were on my min...,1.0,sad
3,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss ...,1.0,love
4,6,downtown,petula clark,1965,when youre alone and life is making you lonely...,1.0,sad


In [128]:
#add decade classification to dataframe
train_songs.loc[(train_songs.Year< 1970) & (train_songs.Year> 1964), 'Decade'] = '1960' 
train_songs.loc[(train_songs.Year< 1980) & (train_songs.Year> 1969), 'Decade'] = '1970'
train_songs.loc[(train_songs.Year< 1990) & (train_songs.Year> 1979), 'Decade'] = '1980'
train_songs.loc[(train_songs.Year< 2000) & (train_songs.Year> 1989), 'Decade'] = '1990'
train_songs.loc[(train_songs.Year< 2010) & (train_songs.Year> 1999), 'Decade'] = '2000'
train_songs.loc[(train_songs.Year< 2020) & (train_songs.Year> 2009), 'Decade'] = '2010'
train_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 5092
Data columns (total 8 columns):
.          428 non-null int64
Song       428 non-null object
Artist     428 non-null object
Year       428 non-null int64
Lyrics     428 non-null object
Source     428 non-null float64
Schmood    428 non-null object
Decade     428 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 30.1+ KB


In [129]:
train_df = train_songs.filter(['Lyrics','Schmood', 'Decade'])
train_df = train_df.reset_index(drop=True)
train_df.head()

,Lyrics,Schmood,Decade
0,sam the sham miscellaneous wooly bully wooly b...,saucy,1960
1,sugar pie honey bunch you know that i love you...,love,1960
2,when i woke up this morning you were on my min...,sad,1960
3,you never close your eyes anymore when i kiss ...,love,1960
4,when youre alone and life is making you lonely...,sad,1960


In [130]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lyrics_clean = []

for index, row in train_df.iterrows():
    song = row['Lyrics']
    tokens = word_tokenize(song)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    str = " "
    tokens = str.join(tokens)
    lyrics_clean.append(tokens)
lyrics_clean
train_df['Clean Lyrics'] = lyrics_clean
train_df.head()

,Lyrics,Schmood,Decade,Clean Lyrics
0,sam the sham miscellaneous wooly bully wooly b...,saucy,1960,sam sham miscellaneous wooly bully wooly bully...
1,sugar pie honey bunch you know that i love you...,love,1960,sugar pie honey bunch know love cant help love...
2,when i woke up this morning you were on my min...,sad,1960,woke morning mind mind got troubles whoaoh got...
3,you never close your eyes anymore when i kiss ...,love,1960,never close eyes anymore kiss lips theres tend...
4,when youre alone and life is making you lonely...,sad,1960,youre alone life making lonely always go downt...


In [155]:
X_train = train_df.loc[:100, 'Clean Lyrics'].values
y_train = train_df.loc[:100, 'Schmood'].values
X_test = train_df.loc[101:, 'Clean Lyrics'].values
y_test = train_df.loc[101:, 'Schmood'].values

In [156]:
#X_train = train_df.loc[:124, 'Clean Lyrics'].values
#y_train = train_df.loc[:124, 'Decade'].values
#X_test = train_df.loc[125:, 'Clean Lyrics'].values
#y_test = train_df.loc[125:, 'Decade'].values

In [157]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(101, 1958) (327, 1958)


In [158]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(train_vectors, y_train)

In [159]:
from  sklearn.metrics  import accuracy_score
predicted = clf.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.20795107033639143
